In [1]:
import duckdb
import numpy as np
import pandas as pd
import plotly.express as px

# pandas display option adjustments
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [15]:
# Using Kaggle's python API to download dataset for project.
kaggle = KaggleApi()
kaggle.authenticate() # See Kaggle API instructions on how to set authentication in .kaggle/kaggle.json
kaggle.dataset_download_file(dataset='dubradave/formula-1-drivers-dataset',
                             file_name='F1DriversDataset.csv',
                             path='/repos/f1-duckdb/data')

True

In [8]:
# Connecting to local DuckDB dataset
con = duckdb.connect(database='../data/FormulaOne.db')

# Sourcing data from DuckDb into pandas dataframe
df = con.execute('''SELECT * FROM FormulaOne_Drivers''').fetch_df()
df.head()


,Driver,Nationality,Seasons,Championships,Race_Entries,Race_Starts,Pole_Positions,Race_Wins,Podiums,Fastest_Laps,Points,Active,Championship Years,Decade,Pole_Rate,Start_Rate,Win_Rate,Podium_Rate,FastLap_Rate,Points_Per_Entry,Years_Active,Champion
0,Carlo Abate,Italy,"[1962, 1963]",0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,False,NaN,1960,0.0,0.000000,0.0,0.0,0.0,0.000000,2,False
1,George Abecassis,United Kingdom,"[1951, 1952]",0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,False,NaN,1950,0.0,1.000000,0.0,0.0,0.0,0.000000,2,False
2,Kenny Acheson,United Kingdom,"[1983, 1985]",0.0,10.0,3.0,0.0,0.0,0.0,0.0,0.0,False,NaN,1980,0.0,0.300000,0.0,0.0,0.0,0.000000,2,False
3,Andrea de Adamich,Italy,"[1968, 1970, 1971, 1972, 1973]",0.0,36.0,30.0,0.0,0.0,0.0,0.0,6.0,False,NaN,1970,0.0,0.833333,0.0,0.0,0.0,0.166667,5,False
4,Philippe Adams,Belgium,[1994],0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,False,NaN,1990,0.0,1.000000,0.0,0.0,0.0,0.000000,1,False


In [22]:
df.dtypes, df.columns

(Driver                 object
 Nationality            object
 Seasons                object
 Championships         float64
 Race_Entries          float64
 Race_Starts           float64
 Pole_Positions        float64
 Race_Wins             float64
 Podiums               float64
 Fastest_Laps          float64
 Points                float64
 Active                   bool
 Championship Years     object
 Decade                  int64
 Pole_Rate             float64
 Start_Rate            float64
 Win_Rate              float64
 Podium_Rate           float64
 FastLap_Rate          float64
 Points_Per_Entry      float64
 Years_Active            int64
 Champion                 bool
 dtype: object,
 Index(['Driver', 'Nationality', 'Seasons', 'Championships', 'Race_Entries',
        'Race_Starts', 'Pole_Positions', 'Race_Wins', 'Podiums', 'Fastest_Laps',
        'Points', 'Active', 'Championship Years', 'Decade', 'Pole_Rate',
        'Start_Rate', 'Win_Rate', 'Podium_Rate', 'FastLap_Rate',
      

### Exploring Driver x Nationality

Using DuckDB to query the F1 Driver Dataset that was importing into memory as a pandas pd

In [9]:
drv_nat = duckdb.sql('''SELECT Count(Distinct(Driver)) as Driver_Cnt, Nationality
                        FROM df
                        GROUP BY Nationality
                        ORDER BY Driver_Cnt DESC''').df()

Using plotly express to plot Driver Counts by Nationality


In [12]:
fig = px.bar(x=drv_nat['Nationality'], y=drv_nat['Driver_Cnt'], text_auto=True)
fig.update_traces(textposition='outside', textfont_size=10)
fig.update_layout(title='Driver Counts by Nationality',
                  yaxis_title='Driver Count',
                  xaxis_title=None,
                  showlegend=False,
                  uniformtext_minsize=3,
                  uniformtext_mode='hide')

fig.show()

Looking at the above chart, it looks like Germany is split into Germany and pre-unification West Germany/East Germany.   
Here, I will highlight the observations mentioned

In [13]:
lowlight = '#CFD8DC'
highlight = '#00796B'
country_highlighter = (
    [highlight if i in ['Germany','West Germany','East Germany','East Germany, West Germany'] 
     else lowlight for i in drv_nat['Nationality']]
)

fig = px.bar(x=drv_nat['Nationality'], y=drv_nat['Driver_Cnt'], 
             text_auto=True, color_discrete_sequence=country_highlighter, color=drv_nat['Nationality'])
fig.update_traces(textposition='outside', textfont_size=10)
fig.update_layout(title='Driver Counts by Nationality - Highlight Germany Versions',
                  yaxis_title='Driver Count',
                  xaxis_title=None,
                  showlegend=False,
                  uniformtext_minsize=3,
                  uniformtext_mode='hide')

fig.show()

Looks like Germany is split into 4 buckets. For the sake of this analysis, Im going to rebucket them into a single Germany node.

In [14]:
drv_nat_one_germany = duckdb.sql('''SELECT 
                                        Count(Distinct(Driver)) as Driver_Cnt, 
                                        CASE WHEN Nationality IN (
                                            'Germany','West Germany','East Germany',
                                            'East Germany, West Germany') THEN 'Germany'
                                             ELSE Nationality
                                             END AS Nationality_Corr
                                    FROM df
                                    GROUP BY Nationality_Corr
                                    ORDER BY Driver_Cnt DESC''').df()


In [15]:
country_highlighter = (
    [highlight if i in ['Germany','West Germany','East Germany','East Germany, West Germany'] 
     else lowlight for i in drv_nat_one_germany['Nationality_Corr']]
)

fig = px.bar(x=drv_nat_one_germany['Nationality_Corr'], y=drv_nat_one_germany['Driver_Cnt'], 
             text_auto=True, color_discrete_sequence=country_highlighter, color=drv_nat_one_germany['Nationality_Corr'])
fig.update_traces(textposition='outside', textfont_size=10)
fig.update_layout(title='Driver Counts by Nationality - One Germany',
                  yaxis_title='Driver Count',
                  xaxis_title=None,
                  showlegend=False,
                  uniformtext_minsize=3,
                  uniformtext_mode='hide')

fig.show()